## Computation of the table of sporadic (nondegenerate) tetrahedra

Tested using SageMath 9.1.

In [3]:
import re
from itertools import permutations

Read in the computed list of sporadic examples of tetrahedra with rational dihedral angles.

In [10]:
sols = []
with open('DATA/sporadics_genuine_tetrahedra.txt') as f:
    for s in f:
        m = re.match(r"(\[.*\])", s)
        if m:
            sols.append(sage_eval(m.group(1)))
print(len(sols))

76


Verify that each of these is actually a solution.

In [4]:
for (N, n1, n2, n3, n4, n5, n6) in sols:
    K.<z> = CyclotomicField(2*N)
    (x1, x2, x3, x4, x5, x6) = tuple(z^i for i in (n1, n2, n3, n4, n5, n6))
    gram_matrix = -Matrix([[-2, x1+1/x1, x3+1/x3, x5+1/x5],
                       [x1+1/x1, -2, x6+1/x6, x4+1/x4],
                       [x3+1/x3, x6+1/x6, -2, x2+1/x2],
                       [x5+1/x5, x4+1/x4, x2+1/x2, -2]])
    assert(gram_matrix.det() == 0)

Provide citations from the literature (Boltyanskii's book).

In [5]:
labels = {(12, (6, 4, 6, 4, 3, 6)): r'T_0',
          (12, (8, 6, 4, 3, 3, 4)): r'H_2(\pi/4)',
          (15, (9, 5, 6, 6, 5, 5)): r'T_{23}',
          (15, (9, 9, 5, 5, 3, 3)): r'T_7',
          (15, (10, 10, 5, 3, 3, 3)): r'T_{18}',
          (30, (15, 12, 12, 10, 12, 10)): r'T_5',
          (30, (15, 12, 15, 10, 6, 15)): r'T_1',
          (30, (18, 6, 15, 15, 10, 10)): r'T_{13}',
          (30, (18, 12, 15, 10, 6, 10)): r'T_{21}',
          (30, (20, 10, 15, 10, 6, 12)): r'T_{17}',
          (30, (18, 15, 12, 6, 10, 10)): r'T_3',
          (30, (20, 10, 18, 6, 6, 15)): r'T_4',
          (30, (20, 15, 12, 10, 6, 6)): r'T_{16}',
          (30, (20, 15, 12, 6, 6, 12)): r'T_2',
          (30, (24, 15, 10, 6, 6, 10)): r'T_6'
          }

Normalize each solution within its $S_4$-orbit, to pick the element which is lexicographically smallest (and update the labels directory accordingly).
Then resort the whole list and remove duplicates.

In [6]:
d = {(0,1): 1, (1,0): 1, (2,3): 2, (3,2): 2, (0,2): 3, (2,0): 3, (1,3): 4, (3,1): 4, (0,3): 5, (3,0): 5, (1,2): 6, (2,1): 6}
s4_orbits = {}
sols2 = []
for v in sols:
    tmp = []
    for t in permutations(range(4)):
        tmp.append((v[0], (v[d[t[0],t[1]]], v[d[t[2],t[3]]], v[d[t[0],t[2]]], v[d[t[1],t[3]]], v[d[t[0],t[3]]], v[d[t[1],t[2]]])))
    tmp.sort()
    sols2.append(tmp[0])
    s4_orbits[tmp[0]] = tmp
    for t in tmp:
        if t in labels:
            labels[tmp[0]] = labels[t]
sols2 = sorted(set(sols2))
print(len(sols2))

59


Group solutions into orbits for the group generated by $S_4$ and Regge symmetries. We do this by finding connected components of a Cayley graph.

In [7]:
def regge(t):
    (a,b,c,d,e,f) = t
    s = (c+d+e+f)/2
    return(a,b,s-c,s-d,s-e,s-f)

In [12]:
syms = (regge, # Regge symmetry
       (lambda t: (t[0],t[1],t[5],t[4],t[3],t[2])), # Transposition (1, 2)
       (lambda t: (t[2],t[3],t[0],t[1],t[4],t[5])), # Transposition (2, 3)
       )

In [13]:
edges = []
vertices = []
queue = sols2[:]
while queue:
    (N,s) = queue.pop(0)
    vertices.append((N,s))
    v1 = tuple(x/N for x in s)
    for g in syms:
        v2 = g(v1)
        N2 = lcm(x.denominator() for x in v2)
        s2 = tuple(x*N2 for x in v2)
        edges.append([(N,s), (N2,s2)])
        if (N2,s2) not in (vertices + queue):
            queue.append((N2,s2))
        assert(N2 <= 60)
Gamma = Graph(edges, loops=True)

Locate our chosen $S_4$-orbit representatives among the $W(D_6)$-orbits. Also, make sure that we did not produce anything extra.

In [15]:
l = Gamma.connected_components()
l2 = []
for cc in l:
    cc2 = []
    for x in cc:
        if x in sols2:
            cc2.append(sols2.index(x))
        elif all(i>0 for i in x[1]):
            assert any(x in tmp for tmp in s4_orbits.values())
    l2.append(sorted(cc2))
l2.sort()

For each connected component, identify the "special" representative which has the least sum of denominators when written in reduced form, and sort that first.

In [16]:
for cc in l2:
    d = [(sols2[i][0], sum((k/sols2[i][0]).denominator() for k in sols2[i][1]), i) for i in cc]
    d.sort()
    t1 = d[0][2]
    cc.remove(t1)
    cc.insert(0, t1)

Print the lists of connected components in raw form.

In [17]:
for cc in l2:
    print([sols2[i] for i in cc])

[(12, (3, 4, 3, 4, 6, 8)), (24, (5, 9, 6, 8, 13, 15))]
[(12, (3, 6, 4, 6, 4, 6)), (24, (7, 11, 7, 13, 8, 12))]
[(15, (3, 3, 3, 5, 10, 10)), (15, (2, 4, 4, 4, 10, 10)), (15, (3, 3, 4, 4, 9, 11))]
[(15, (3, 3, 5, 5, 9, 9))]
[(15, (5, 5, 5, 9, 6, 6)), (15, (3, 7, 6, 6, 7, 7)), (15, (4, 8, 5, 5, 7, 7))]
[(21, (3, 9, 7, 7, 12, 12)), (21, (4, 10, 6, 6, 12, 12)), (21, (6, 6, 7, 7, 9, 15))]
[(30, (6, 12, 10, 15, 10, 20)), (30, (4, 14, 10, 15, 12, 18)), (60, (8, 28, 19, 31, 25, 35)), (60, (12, 24, 15, 35, 25, 35)), (60, (13, 23, 15, 35, 24, 36)), (60, (13, 23, 19, 31, 20, 40))]
[(30, (6, 18, 10, 10, 15, 15)), (30, (4, 16, 12, 12, 15, 15)), (30, (9, 21, 10, 10, 12, 12))]
[(30, (6, 6, 10, 12, 15, 20)), (30, (5, 7, 11, 11, 15, 20)), (60, (7, 17, 20, 24, 35, 35)), (60, (7, 17, 22, 22, 33, 37)), (60, (10, 14, 17, 27, 35, 35)), (60, (12, 12, 17, 27, 33, 37))]
[(30, (6, 10, 10, 15, 12, 18)), (30, (5, 11, 10, 15, 13, 17)), (60, (10, 22, 21, 29, 25, 35)), (60, (11, 21, 19, 31, 26, 34)), (60, (11, 21, 21

Print the lists of connected components in LaTeX, including labels when available.

In [18]:
print(r"\begin{tabular}{c|c}")
print(r"$N$ & $\frac{N}{\pi}(\theta_{12}, \theta_{34}, \theta_{13}, \theta_{24}, \theta_{14}, \theta_{23})$ as multiples of $\pi/N$ \\")
for cc in l2:
    print("\\hline")
    for N in sorted(set(sols2[i][0] for i in cc)):
        outlist = ["$" + str(sols2[i][1])+(' = {}$'.format(labels[sols2[i]]) if sols2[i] in labels else "$") for i in cc if sols2[i][0] == N]
        print(r"{} & ".format(N), end='')
        if len(outlist) <= 3:
            print(", ".join(outlist), end='')
        elif len(outlist) == 4:
            print(", ".join(outlist[:2]) + ", \\\\& " + ", ".join(outlist[2:]))
        elif len(outlist) == 6:
            print(", ".join(outlist[:3]) + ", \\\\& " + ", ".join(outlist[3:]))
        else:
            raise ValueError
        print(r"\\")
print(r"\end{tabular}")

\begin{tabular}{c|c}
$N$ & $\frac{N}{\pi}(\alpha_{12}, \alpha_{34}, \alpha_{13}, \alpha_{24}, \alpha_{14}, \alpha_{23})$  \\
\hline
12 & $(3, 4, 3, 4, 6, 8) = H_2(\pi/4)$\\
24 & $(5, 9, 6, 8, 13, 15)$\\
\hline
12 & $(3, 6, 4, 6, 4, 6) = T_0$\\
24 & $(7, 11, 7, 13, 8, 12)$\\
\hline
15 & $(3, 3, 3, 5, 10, 10) = T_{18}$, $(2, 4, 4, 4, 10, 10)$, $(3, 3, 4, 4, 9, 11)$\\
\hline
15 & $(3, 3, 5, 5, 9, 9) = T_7$\\
\hline
15 & $(5, 5, 5, 9, 6, 6) = T_{23}$, $(3, 7, 6, 6, 7, 7)$, $(4, 8, 5, 5, 7, 7)$\\
\hline
21 & $(3, 9, 7, 7, 12, 12)$, $(4, 10, 6, 6, 12, 12)$, $(6, 6, 7, 7, 9, 15)$\\
\hline
30 & $(6, 12, 10, 15, 10, 20) = T_{17}$, $(4, 14, 10, 15, 12, 18)$\\
60 & $(8, 28, 19, 31, 25, 35)$, $(12, 24, 15, 35, 25, 35)$, \\& $(13, 23, 15, 35, 24, 36)$, $(13, 23, 19, 31, 20, 40)$
\\
\hline
30 & $(6, 18, 10, 10, 15, 15) = T_{13}$, $(4, 16, 12, 12, 15, 15)$, $(9, 21, 10, 10, 12, 12)$\\
\hline
30 & $(6, 6, 10, 12, 15, 20) = T_{16}$, $(5, 7, 11, 11, 15, 20)$\\
60 & $(7, 17, 20, 24, 35, 35)$, $(7, 17, 22